In [1]:
# 경고(warning) 비표시
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

# 제4장 고객의 행동을 예측하는 테크닉 10

### 테크닉 31 : 데이터를 읽어들이고 확인하자

In [6]:
use_log = pd.read_csv('use_log.csv')
use_log.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [7]:
customer = pd.read_csv('customer_join.csv')
customer.isnull().sum()

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

### 테크닉 32 : 클러스터링으로 회원을 그룹화하자

In [16]:
customer_clustering = customer[["mean", "median", "max", "min", "membership_period"]]
customer_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,47
1,5.083333,5.0,7,3,47
2,4.583333,5.0,6,3,47
3,4.833333,4.5,7,2,47
4,3.916667,4.0,6,1,47


In [17]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
customer_clustering_sc = sc.fit_transform(customer_clustering)

In [21]:
kmeans = KMeans(n_cluster=4, random_state=0)
clustering = kmeans.fit(customer_clustering_sc)
customer_clustering["cluster"] = clusters.labels_
print(customer_clustering["cluster"].unique())


TypeError: __init__() got an unexpected keyword argument 'n_cluster'

### 테크닉 33 : 클러스터링 결과를 분석하자

### 테크닉 34 : 클러스터링 결과를 가시화하자

### 테크닉 35 : 클러스터링 결과를 바탕으로 탈퇴회원의 경향을 파악하자

### 테크닉 36 : 다음달의 이용횟수 예측을 위해 테이터를 준비하자

### 테크닉 37 : 특징이 되는 변수를 추가하자

### 테크닉 38 : 다음달 이용횟수를 예측하는 모델을 구축하자

### 테크닉 39 : 모델에 기여하는 변수를 확인하자

### 테크닉 40 : 다음달의 이용횟수를 예측하자